In [101]:
import torch

# Load the model
model_weights = torch.load('A4C_model_seg.pt')

# Print the keys of the loaded model to understand its structure
print(model_weights.keys())


dict_keys(['epoch', 'best_fitness', 'model', 'ema', 'updates', 'optimizer', 'train_args', 'date', 'version'])


In [4]:
import torch
import cv2
from ultralytics import YOLO

# Load the pretrained YOLO model (replace 'yolov8n.pt' with your model path if needed)
model = YOLO('yolov8n.pt')  # Adjust this if you're using a custom YOLO model

# Initialize the camera
cap = cv2.VideoCapture(0)

# Check if the camera opened successfully
if not cap.isOpened():
    print("Error: Could not open camera.")
    exit()

# Loop for real-time object detection
while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame.")
        break

    # Run YOLO object detection on the frame
    results = model(frame)

    # Initialize counters for humans and objects
    human_count = 0
    total_objects = 0

    # Process each detection result
    for result in results:
        # Access each bounding box in the result
        for box in result.boxes:
            # Get the bounding box coordinates, confidence score, and class ID
            x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())  # Convert to integers
            confidence = box.conf[0].item()                  # Convert confidence to float
            class_id = int(box.cls[0].item())                # Convert class ID to integer
            
            # Get the class label and format label text
            label = model.names[class_id]
            label_text = f"{label} {confidence:.2f}"

            # Check if the detected object is a human
            if label.lower() == 'person':  # Assuming 'person' is the class name for humans
                human_count += 1
            total_objects += 1

            # Draw the bounding box
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)  # Green bounding box

            # Draw the label above the bounding box
            cv2.putText(frame, label_text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX,
                        0.5, (0, 255, 0), 2)  # Green label text

    # Display the total counts on the frame
    cv2.putText(frame, f'Total Humans: {human_count}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX,
                1, (0, 255, 0), 2)
    cv2.putText(frame, f'Total Objects: {total_objects}', (10, 60), cv2.FONT_HERSHEY_SIMPLEX,
                1, (0, 255, 0), 2)

    # Display the resulting frame
    cv2.imshow('Real-time Object Detection', frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the camera and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()



0: 384x640 (no detections), 40.5ms
Speed: 2.8ms preprocess, 40.5ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 1 chair, 34.8ms
Speed: 1.2ms preprocess, 34.8ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 1 chair, 36.7ms
Speed: 1.5ms preprocess, 36.7ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 1 chair, 39.7ms
Speed: 1.4ms preprocess, 39.7ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 1 chair, 28.2ms
Speed: 1.4ms preprocess, 28.2ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 1 chair, 24.3ms
Speed: 1.2ms preprocess, 24.3ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 1 chair, 31.0ms
Speed: 1.5ms preprocess, 31.0ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 1 chair, 25.1ms
Speed: 1.1ms prepro

2024-11-07 19:32:55.230 python[1136:16397] +[IMKClient subclass]: chose IMKClient_Modern
2024-11-07 19:32:55.230 python[1136:16397] +[IMKInputSession subclass]: chose IMKInputSession_Modern


0: 384x640 1 person, 1 chair, 27.9ms
Speed: 1.2ms preprocess, 27.9ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 1 chair, 27.6ms
Speed: 1.6ms preprocess, 27.6ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 1 chair, 25.9ms
Speed: 1.2ms preprocess, 25.9ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 1 chair, 29.1ms
Speed: 1.2ms preprocess, 29.1ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 1 chair, 34.6ms
Speed: 1.2ms preprocess, 34.6ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 1 chair, 24.1ms
Speed: 1.2ms preprocess, 24.1ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 1 chair, 29.3ms
Speed: 1.3ms preprocess, 29.3ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 1 chair, 26.1ms
Speed: 1.1ms prepr

# Assistant
The error you're encountering indicates that the model you're trying to load contains a layer (`DepthwiseConv2D`) that is not recognizing the keyword argument `groups`. This can happen if there is a version mismatch between the Keras model you saved and the Keras (or TensorFlow) version you are currently using to load the model.

To fix this error, you can try the following steps:

1. **Check TensorFlow/Keras Version**: Ensure that the version of TensorFlow/Keras you are using to load the model matches the version used to create and save the model. You can check your current version with:
   

In [59]:
   import tensorflow as tf
   print(tf.__version__)
   

2.18.0



2. **Update TensorFlow/Keras**: If there is a version mismatch, consider updating TensorFlow/Keras to the version that was used to create the model. You can do this using pip:
   

bash
   pip install --upgrade tensorflow
   


3. **Modify the Model**: If you have access to the original model code, you can try to modify the model to remove or adjust the `groups` parameter in the `DepthwiseConv2D` layer.

Would you like me to provide a code snippet to check your TensorFlow version?

In [61]:
!pip install --upgrade tensorflow
